# Step1: Uploading ufo data from S3

In [1]:

import pandas as pd
import numpy as np
from datetime import datetime

import boto3
from sagemaker import get_execution_role
import sagemaker.amazon.common as smac

In [5]:

role = get_execution_role()
bucket = 'ml-lab-ufo-elly'
prefix = 'ufo_dataset'
data_key = 'ufodata.csv'
data_location = 's3://{}/{}/{}'.format(bucket, prefix, data_key)

df = pd.read_csv(data_location, low_memory=False)

In [2]:

role = get_execution_role()
bucket = 'ml-lab-ufo-elly'
prefix = 'ufo_dataset'
data_key = 'ufodata.csv'
data_location = 's3://{}/{}/{}'.format(bucket, prefix, data_key)

df = pd.read_csv(data_location, low_memory=False)

In [6]:
df.head()

,reportedTimestamp,eventDate,eventTime,shape,duration,witnesses,weather,firstName,lastName,latitude,longitude,sighting,physicalEvidence,contact,researchOutcome
0,1977-04-04T04:02:23.340Z,3/31/1977,23:46,circle,4,1,rain,Ila,Bashirian,47.329444,-122.578889,Y,N,N,explained
1,1982-11-22T02:06:32.019Z,11/15/1982,22:04,disk,4,1,partly cloudy,Eriberto,Runolfsson,52.664913,-1.034894,Y,Y,N,explained
2,1992-12-07T19:06:52.482Z,12/7/1992,19:01,circle,49,1,clear,Miller,Watsica,38.951667,-92.333889,Y,N,N,explained
3,2011-02-24T21:06:34.898Z,2/21/2011,20:56,disk,13,1,partly cloudy,Clifton,Bechtelar,41.496944,-71.367778,Y,N,N,explained
4,1991-03-09T16:18:45.501Z,3/9/1991,11:42,circle,17,1,mostly cloudy,Jayda,Ebert,47.606389,-122.330833,Y,N,N,explained


In [8]:
df.shape

(18000, 15)

# Step2: Cleaning. transforming, and preparing the data

In [9]:
df_geo = df[['latitude','longitude']]

In [10]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   18000 non-null  float64
 1   longitude  18000 non-null  float64
dtypes: float64(2)
memory usage: 281.4 KB


In [11]:
# transform the dataframe to numpy

In [12]:
data_train = df_geo.values.astype('float32')
data_train

array([[  47.329445, -122.57889 ],
       [  52.664913,   -1.034894],
       [  38.951668,  -92.333885],
       ...,
       [  36.86639 ,  -83.888885],
       [  35.385834,  -94.39833 ],
       [  29.883055,  -97.94111 ]], dtype=float32)

# step3: Create and train model

In [13]:
from sagemaker import KMeans

num_cluster = 10
output_location = 's3://'+ bucket +'/mdole-artifacts'

km = KMeans(role = role, 
           train_instance_count = 1,
           train_instance_type ='ml.c4.xlarge',
           output_path = output_location,
           k = num_cluster)

In [14]:
job_name = 'kmeans-geo-job-{}'.format(datetime.now().strftime("%Y%m%d%H%M"))
print('Here is the job name{}'.format(job_name))

Here is the job namekmeans-geo-job-202007122225


In [15]:
%%time
km.fit(km.record_set(data_train), job_name=job_name)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-07-12 22:25:06 Starting - Starting the training job...
2020-07-12 22:25:08 Starting - Launching requested ML instances.........
2020-07-12 22:26:46 Starting - Preparing the instances for training......
2020-07-12 22:28:04 Downloading - Downloading input data...
2020-07-12 22:28:22 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/12/2020 22:28:46 INFO 139815496828736] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'_enable_profiler': u'false', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'local_lloyd_num_trials': u'auto', u'_log_level': u'info', u'_kvstore': u'auto', u'local_lloyd_init_method': u'kmeans++', u'force_dense': u'true', u'epochs': u'1', u'init_method': u'random', u'local_lloyd_tol': u'0.0001', u'local_lloyd_max_iter': u'300', u'_disable_wait_to_read': u'false', u'extra_center_factor': u'auto', 

In [17]:
bucket

'ml-lab-ufo-elly'

In [18]:
import os

model_key ='mdole-artifacts/'+ job_name + '/output/model.tar.gz'

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

2304

In [20]:
!pip install mxnet

     |████████████████████████████████| 68.7 MB 58 kB/s s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [21]:
import mxnet as mx
Kmeans_model_params = mx.ndarray.load('model_algo-1')

In [22]:
cluster_centroids_kmeans = pd.DataFrame(Kmeans_model_params[0].asnumpy())
cluster_centroids_kmeans.columns=df_geo.columns
cluster_centroids_kmeans

,latitude,longitude
0,46.918221,-120.526367
1,52.126045,0.378868
2,30.734335,-81.900154
3,41.393799,-74.457085
4,-10.968700,128.140167
5,35.291668,-98.063751
6,35.608452,-117.294327
7,21.187809,47.924477
8,41.041370,-86.496300
9,43.195156,-153.404190


In [25]:
from io import StringIO

In [29]:
csv_buffer = StringIO()
cluster_centroids_kmeans.to_csv(csv_buffer, index = False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'results/ten_locations_kmeans.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '07ECFC5B642294C7',
  'HostId': 'YygLvCRT/J4AZUTEzriktCLl5t3eQp2eFtxqFyho/5TGGH7kJESlzDc3TJpNbDw3//4QdMznkTg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'YygLvCRT/J4AZUTEzriktCLl5t3eQp2eFtxqFyho/5TGGH7kJESlzDc3TJpNbDw3//4QdMznkTg=',
   'x-amz-request-id': '07ECFC5B642294C7',
   'date': 'Wed, 08 Jul 2020 00:00:10 GMT',
   'etag': '"04bdaf2b43f4cb94cd115e1595e73101"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"04bdaf2b43f4cb94cd115e1595e73101"'}